used with [domain_adaptation_1](./domain_adaptation_1.ipynb)

## get the data

In [ ]:
# TODO use command line args or someting easier than throwing it here
basedir = '/tigress/kendrab/21032023/'
readpaths = []
noplasmoids_dir = '/tigress/kendrab/06022023/'
noplasmoids_paths = []

for i in range(10):
    totdir = basedir+str(i)+'/'
    for j in range(5,60,5):
        readpaths.append(totdir+f"100samples_idx{j}_bxbybzjyvzexeyez.hdf5")
        
for j in range(5,55,5):
        noplasmoids_paths.append(noplasmoids_dir+f"100samples_idx{j}_bxbybzjyvzexeyez.hdf5")
        
idx_list = []  # to keep track of which file what sample came from
s_list = []
bx_list = []
by_list = []
bz_list = []
ex_list = []
ey_list = []
ez_list = []
jy_list = []
x0_list = []
x1_list = []
topo_list = []

train_idx = None
# import the messy current sheet data
for idx, filepath in enumerate(readpaths):
    with h5py.File(filepath, 'r') as file:
        idx_list += [np.array([idx for i in bx]) for bx in file['bx_mms_smooth'][:]]  # check this structure!!!
        s_list += list(file['s'][:])
        bx_list += list(file['bx_mms_smooth'][:])
        by_list += list(file['by_mms'][:])
        bz_list += list(file['bz_mms_smooth'][:])
        ex_list += list(file['ex_mms'][:]) 
        ey_list += list(file['ey_mms'][:])
        ez_list += list(file['ez_mms'][:])  # vx_mms is simulation vz  thus the filename 
        jy_list += list(file['jy_mms'][:])
        x0_list += list(file['x_mms'][:])
        x1_list += list(file['z_mms'][:])
        topo_list_tmp = list(file['topo'][:])
        for i in range(len(topo_list_tmp)):  # I tried to vectorize this but I didn't get it to work
            topo_list_tmp[i] = torch.from_numpy(topo_list_tmp[i].astype(int) % 2)  # cat 0,2 are not plasmoids, cat 1,3 are
            topo_list_tmp[i] = one_hot(topo_list_tmp[i], num_classes=2)
        topo_list += topo_list_tmp
        
# import the smooth current sheet data
for idx, filepath in enumerate(noplasmoids_paths):
    with h5py.File(filepath, 'r') as file:
        idx_list += [np.array([idx for i in bx]) for bx in file['bx_mms_smooth'][:]]  # check this structure!!!
        s_list += list(file['s'][:])
        bx_list += list(file['bx_mms_smooth'][:])
        by_list += list(file['by_mms'][:])
        bz_list += list(file['bz_mms_smooth'][:])
        ex_list += list(file['ex_mms'][:]) 
        ey_list += list(file['ey_mms'][:])
        ez_list += list(file['ez_mms'][:])  # vx_mms is simulation vz  thus the filename 
        jy_list += list(file['jy_mms'][:])
        x0_list += list(file['x_mms'][:])
        x1_list += list(file['z_mms'][:])
        topo_list_tmp = list(file['topo'][:])
        # FIX THIS
        for i in range(len(topo_list_tmp)):  # I tried to vectorize this but I didn't get it to work
            topo_list_tmp[i] = torch.from_numpy(topo_list_tmp[i].astype(int) % 2)  # cat 0,2 are not plasmoids, cat 1,3 are
            topo_list_tmp[i] = one_hot(topo_list_tmp[i], num_classes=2)
        topo_list += topo_list_tmp

## chunk the data and put in the right format

In [ ]:
idx = batch_unpadded_subsects(idx_list, padding_length, stride)
s = batch_subsects(s_list, input_length, stride)  # not going through training so don't need to shape right
bx = np.expand_dims(batch_subsects(bx_list, input_length, stride),1)
by = np.expand_dims(batch_subsects(by_list, input_length, stride),1)
bz = np.expand_dims(batch_subsects(bz_list, input_length, stride),1)
ex = np.expand_dims(batch_subsects(ex_list, input_length, stride),1)
ey = np.expand_dims(batch_subsects(ey_list, input_length, stride),1)
ez = np.expand_dims(batch_subsects(ez_list, input_length, stride),1)
jy = np.expand_dims(batch_subsects(jy_list, input_length, stride),1)
x0 = batch_unpadded_subsects(x0_list, padding_length, stride)
x1 = batch_unpadded_subsects(x1_list, padding_length, stride)
topo = np.swapaxes(batch_unpadded_subsects(topo_list, padding_length, stride), 1, 2)

print(bx.shape)
print(topo.shape)
# shuffle the segments so they aren't adjacent to overlapping/similar segments NOT NEEDED bc dataloader shuffles
# idx, s, bx, by, bz, ex, ey, ez, jy, x0, x1, topo = shuffle(idx, s, bx, by, bz, ex, ey, ez, jy, x0, x1, topo)

# numpy arrays to torch tensors (while crying about how many lines of code this is surely there is a better way)
idx = torch.from_numpy(idx).to(device, dtype=dtype)
s = torch.from_numpy(s).to(device, dtype=dtype)
bx = torch.from_numpy(bx).to(device, dtype=dtype)
by = torch.from_numpy(by).to(device, dtype=dtype)
bz = torch.from_numpy(bz).to(device, dtype=dtype)
ex = torch.from_numpy(ex).to(device, dtype=dtype)
ey = torch.from_numpy(ey).to(device, dtype=dtype)
ez = torch.from_numpy(ez).to(device, dtype=dtype)
jy = torch.from_numpy(jy).to(device, dtype=dtype)
x0 = torch.from_numpy(x0).to(device, dtype=dtype)
x1 = torch.from_numpy(x1).to(device, dtype=dtype)
topo = torch.from_numpy(topo).to(device, dtype=dtype)
print(bx.dtype)

# collect data into Datasets
sim_dset = TensorDataset(idx, s, bx, by, bz, ex, ey, ez, jy, x0, x1, topo)
# Make DataLoaders for the training and test data
sim_dl = DataLoader(sim_dset, batch_size = batch_size, shuffle=True, drop_last=True)

### Garbage collection?

In [ ]:
for item in topo_list:
    del item
    
for item in topo_list_tmp:
    del item
    
del topo_list
del topo_list_tmp